In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

url = 'https://www.gnu.org/people/people.en.html'
css_selector = 'h4'
response = requests.get(url)

In [5]:
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
raw_inner_texts = [elem.get_text() for elem in soup.select(css_selector)]
df = pd.DataFrame(raw_inner_texts, columns=['raw_inner_texts'])
df

,raw_inner_texts
0,Adam Bilbrough \n<abilbrou@gmail.com>
1,Adam Fedor <fedor@gnu.org>
2,Adam Spiers
3,Adrienne G. Thompson <agt@codeartnow.com>
4,Akim\nDemaille
...,...
245,Yann Dirson
246,Yngve Svendsen
247,Yoni Rabkin (aka yrk)
248,Yuchen Pei


In [12]:
# re.sub('\n', ' ', 'The quick brown fox') 
# re.sub(' {2,}', ' ', 'The quick brown fox') 
df.raw_inner_texts = df.raw_inner_texts.str.replace('\n', ' ')
df.raw_inner_texts = df.raw_inner_texts.str.replace('  ', ' ')
df

,raw_inner_texts
0,Adam Bilbrough <abilbrou@gmail.com>
1,Adam Fedor <fedor@gnu.org>
2,Adam Spiers
3,Adrienne G. Thompson <agt@codeartnow.com>
4,Akim Demaille
...,...
245,Yann Dirson
246,Yngve Svendsen
247,Yoni Rabkin (aka yrk)
248,Yuchen Pei
